In [1]:
import sys, os, glob, yaml

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import pprint
import seaborn as sns
import trackml.dataset

In [4]:
import torch
from torch_geometric.data import Data
import itertools

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
os.environ['EXATRKX_DATA'] = os.path.abspath(os.curdir)

## _Classifier Evaluation_

### _(+) - Test Dataset_

In [7]:
inputdir="run/gnn_evaluation/test"
testset = sorted(glob.glob(os.path.join(inputdir, "*")))

In [8]:
event_idx = 1

In [9]:
feature_data = torch.load(testset[event_idx], map_location=device)

In [10]:
feature_data

Data(x=[171, 3], pid=[171], layers=[171], event_file='/global/cscratch1/sd/aakram/train_all/event0000095001', hid=[171], pt=[171], modulewise_true_edges=[2, 160], layerwise_true_edges=[2, 172], edge_index=[2, 808], y_pid=[808], scores=[1616])

### _(+) - Load Model_

- Load model checkpoint
- Load saved config file

In [11]:
from LightningModules.GNN.Models.interaction_gnn import InteractionGNN

In [12]:
ckpnt_path = "run/lightning_models/lightning_checkpoints/GNNStudy/version_1/checkpoints/last.ckpt"

In [13]:
# Model Checkpoint
checkpoint = torch.load(ckpnt_path, map_location=device)

In [14]:
# Checkpointing Hyperparameters
print(checkpoint["hyper_parameters"])

{'input_dir': '/global/cscratch1/sd/aakram/run/feature_store', 'output_dir': '/global/cscratch1/sd/aakram/run/gnn_evaluation', 'project': 'GNNStudy', 'edge_cut': 0.5, 'pt_signal_min': 0.0, 'pt_background_min': 0.0, 'datatype_names': ['train', 'val', 'test'], 'datatype_split': [90000, 5000, 5000], 'noise': False, 'overwrite': True, 'spatial_channels': 3, 'cell_channels': 0, 'hidden': 128, 'n_graph_iters': 8, 'nb_node_layer': 3, 'nb_edge_layer': 3, 'emb_channels': 0, 'layernorm': True, 'aggregation': 'sum_max', 'hidden_activation': 'ReLU', 'weight': 2, 'warmup': 200, 'lr': 0.001, 'factor': 0.3, 'patience': 10, 'regime': 'pid', 'max_epochs': 40, 'callbacks': ['GNNTelemetry', 'GNNBuilder'], 'resume_id': 'version_1', 'logger': 'tb', 'model_library': 'LightningModules', 'artifact_library': '/global/cscratch1/sd/aakram/run/lightning_models/lightning_checkpoints', 'set': 'GNN', 'name': 'InteractionGNN', 'config': 'train_quickstart_GNN.yaml', 'batch_config': 'batch_gpu_default.yaml', 'inference

In [15]:
# get the checkpoint config
config = checkpoint["hyper_parameters"]

In [23]:
# Local Modifications
config["checkpoint_path"] = ckpnt_path
config["input_dir"] = "run/feature_store"
config["output_dir"] = "run/gnn_evaluation"
config["artifact_library"] = "lightning_models/lightning_checkpoints"
config["datatype_split"] = [90000, 5000, 10]
config["callbacks"] = ['GNNTelemetry']

In [24]:
print("hparams:\n", checkpoint["hyper_parameters"])

hparams:
 {'input_dir': 'run/feature_store', 'output_dir': 'run/gnn_evaluation', 'project': 'GNNStudy', 'edge_cut': 0.5, 'pt_signal_min': 0.0, 'pt_background_min': 0.0, 'datatype_names': ['train', 'val', 'test'], 'datatype_split': [90000, 5000, 10], 'noise': False, 'overwrite': True, 'spatial_channels': 3, 'cell_channels': 0, 'hidden': 128, 'n_graph_iters': 8, 'nb_node_layer': 3, 'nb_edge_layer': 3, 'emb_channels': 0, 'layernorm': True, 'aggregation': 'sum_max', 'hidden_activation': 'ReLU', 'weight': 2, 'warmup': 200, 'lr': 0.001, 'factor': 0.3, 'patience': 10, 'regime': 'pid', 'max_epochs': 40, 'callbacks': ['GNNTelemetry'], 'resume_id': 'version_1', 'logger': 'tb', 'model_library': 'LightningModules', 'artifact_library': '/global/cscratch1/sd/aakram/run/lightning_models/lightning_checkpoints', 'set': 'GNN', 'name': 'InteractionGNN', 'config': 'train_quickstart_GNN.yaml', 'batch_config': 'batch_gpu_default.yaml', 'inference': False, 'checkpoint_path': 'run/lightning_models/lightning_c

In [25]:
# Init the InteractionGNN
model = InteractionGNN(config)

In [26]:
import pytorch_lightning as pl

In [27]:
from LightningModules.GNN.Models.inference import GNNTelemetry

In [28]:
# load trainer
trainer = pl.Trainer(max_epochs=1, enable_checkpointing=True)
results = trainer.test(model=model, verbose=True)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Testing: 0it [00:00, ?it/s]

In [29]:
results

[{}]